In [33]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze1", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@storage999.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze1_adls = adls_paths["bronze1"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

StatementMeta(hourpool, 10, 10, Finished, Available, Finished)

In [34]:
adls_paths

StatementMeta(hourpool, 10, 11, Finished, Available, Finished)

{'bronze1': 'abfss://bronze1@storage999.dfs.core.windows.net/',
 'silver': 'abfss://silver@storage999.dfs.core.windows.net/',
 'gold': 'abfss://gold@storage999.dfs.core.windows.net/'}

In [35]:
import requests
import json
from datetime import date, timedelta

StatementMeta(hourpool, 10, 12, Finished, Available, Finished)

In [36]:
'''
# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)
end_date = date.today()
'''

StatementMeta(hourpool, 10, 13, Finished, Available, Finished)

In [37]:
start_date, end_date

StatementMeta(hourpool, 10, 14, Finished, Available, Finished)

(datetime.date(2025, 6, 23), datetime.date(2025, 6, 24))

**Download data from API** (Data is in JSON format)

In [38]:
# Construct the API URL with start and end dates provided by Data Factory
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze1_adls}/{start_date}_earthquake_data.json"

        # Convert data to JSON string
        json_data = json.dumps(data, indent=4)

        # Write the JSON data to ADLS
        # Create an RDD with the JSON string and parallelize it
        rdd = spark.sparkContext.parallelize([json_data])

        # Convert RDD to DataFrame and write to ADLS
        df = spark.read.json(rdd)
        df.limit(3) # To speed up processing for tutorial
        df.write.mode("overwrite").json(file_path)

        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

StatementMeta(hourpool, 10, 15, Finished, Available, Finished)

Data successfully saved to abfss://bronze1@storage999.dfs.core.windows.net//2025-06-23_earthquake_data.json


In [39]:
data[0]

StatementMeta(hourpool, 10, 16, Finished, Available, Finished)

{'type': 'Feature',
 'properties': {'mag': 0.9,
  'place': '36 km S of Denali National Park, Alaska',
  'time': 1750723149788,
  'updated': 1750723286782,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak025805k0ua',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak025805k0ua&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 12,
  'net': 'ak',
  'code': '025805k0ua',
  'ids': ',ak025805k0ua,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.55,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 0.9 - 36 km S of Denali National Park, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-151.7572, 63.2146, 8.6]},
 'id': 'ak025805k0ua'}

In [40]:
import json

# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "bronze1_adls": bronze1_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
bronze1_output = json.dumps(output_data)

# Use mssparkutils.notebook.exit() to pass the JSON output to the pipeline
mssparkutils.notebook.exit(bronze1_output)

StatementMeta(hourpool, 10, 17, Finished, Available, Finished)

ExitValue: {"start_date": "2025-06-23", "bronze1_adls": "abfss://bronze1@storage999.dfs.core.windows.net/", "silver_adls": "abfss://silver@storage999.dfs.core.windows.net/", "gold_adls": "abfss://gold@storage999.dfs.core.windows.net/"}